In [ ]:
!pip install langgraph
!pip install langchain-openai
!pip install langchain_community
!pip install langchain tavily-python

In [ ]:
import os
from getpass import getpass
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API key: ")

In [ ]:
import langgraph
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain.tools.tavily_search import TavilySearchResults

In [ ]:
''' We will use the `TavilySearchResults` tool to perform real-time web searches.'''

tool = TavilySearchResults(max_results=4) #increased number of results
print(type(tool))
print(tool.name)

In [ ]:
''' Defining an agent state 
    - To track full conversation history
    - Automatically merge messages across steps (e.g., LLM → tool → LLM)
    - Used by every node in the LangGraph to get/update the state'''

class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

In [ ]:
''' Defining an AI agent that:

- Uses a language model to generate thoughts and actions.

- Executes actions (i.e. tools) if requested by the model.

- Loops back with the results (observations) to inform the final answer.


exists_action(self, state)
- Checks if the latest LLM message contains tool calls.
- Used to decide whether to run action or end the graph

call_openai(self, state)
- Sends all accumulated messages to the LLM, including an optional system prompt.
- Returns the assistant’s next message.

take_action(self, state)
- Executes tools requested by the LLM:
- Iterates over tool_calls
- Looks up each tool by name
- Invokes it with the provided args-
- Returns the results as ToolMessages


'''

class Agent:

    def __init__(self, model, tools, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges(
            "llm",
            self.exists_action,
            {True: "action", False: END}
        )
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile()
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            if not t['name'] in self.tools:      # check for bad tool name from LLM
                print("\n ....bad tool name....")
                result = "bad tool name, retry"  # instruct LLM to retry if bad
            else:
                result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

In [ ]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""

model = ChatOpenAI(model="gpt-3.5-turbo")  #reduce inference cost
abot = Agent(model, [tool], system=prompt)

In [ ]:
conda install -c conda-forge pygraphviz


In [ ]:
from IPython.display import Image

Image(abot.graph.get_graph().draw_png())

In [ ]:
messages = [HumanMessage(content="What is the weather in sf?")]
result = abot.graph.invoke({"messages": messages})

In [ ]:
result

In [ ]:
result['messages'][-1].content

In [ ]:
query = "Who won the super bowl in 2024? In what state is the winning team headquarters located? \
What is the GDP of that state? Answer each question." 
messages = [HumanMessage(content=query)]

model = ChatOpenAI(model="gpt-4o")  # requires more advanced model
abot = Agent(model, [tool], system=prompt)
result = abot.graph.invoke({"messages": messages})

In [ ]:
print(result['messages'][-1].content)